# Проект по определению токсичности комментариев

Цель работы заключается в обучении модели классификации комментариев на позитивные и негативные. Требуется построить модель со значением метрики качества *F1* не меньше 0.75.

**Описание данных**
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Загрузим все необходимые библиотеки

In [1]:
!pip install catboost --quiet

In [2]:
import re
import time


import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score as f1


from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandrkozuhov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aleksandrkozuhov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aleksandrkozuhov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrkozuhov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aleksandrkozuhov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Выгрузим данные

In [3]:
df = pd.read_csv('toxic_comments.csv', index_col=[0])

Рассмотрим имеющиеся данные

In [4]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Проверим данные на дубликаты

In [7]:
df.duplicated().sum()

0

Отлично, дубликатов нет. Лемматизируем текст с помощью WordNetLemmatizer из библиотеки NLTK.

In [8]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()
def lemmatize(sentence):
    sentence = sentence.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', sentence)
    word_list = nltk.word_tokenize(text)
    lemmatized_output = " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return " ".join(lemmatized_output.split())



In [9]:
df['lemm_text'] = df['text'].apply(lemmatize)

In [10]:
df.sample(5)

,text,toxic,lemm_text
47767,you will be blocked from editing Wikipedia.,0,you will be block from edit wikipedia
11183,"""\n\n Guy de Lusignan \n\nCome on Adam, I was ...",0,guy de lusignan come on adam i be surprised by...
54598,"""\n\n HI! \n\nYess its me Pralph, I was gonna ...",0,hi yes it me pralph i be gon na authros some s...
149886,"""\nThe addition was a """"wikipedia situation wh...",0,the addition be a wikipedia situation where a ...
32833,"), and created a Philosophy lecturer sub-secti...",0,and create a philosophy lecturer sub section u...


Далее следует векторизировать данные для работы. Для этого рассчитаем величины TF-IDF, но перед этим разделим данные на тренировочную и тестовую выборки для кросс-валидации

In [11]:
df_tr, df_new = train_test_split(df, test_size=.2, random_state=12345, stratify=df['toxic'])

In [12]:
train, test = train_test_split(df_new, test_size=.2, random_state=12345)

target_train = train['toxic']
target_test = test['toxic']

count_tf_idf = TfidfVectorizer(stop_words=stopwords.words('english'), ngram_range=(1,2))

train_tf_idf = count_tf_idf.fit_transform(train['lemm_text'])
test_tf_idf = count_tf_idf.transform(test['lemm_text'])

## Обучение

Обучим 2 модели: логистическую регрессию и градиентный бустинг LightGBM

### LogisticRegression

In [18]:
params_reg = {
    'C': [.1,.5,1,2,3,4,5]
}

model_log_reg = LogisticRegression(random_state=12345, max_iter=150, class_weight='balanced')

model_l_r = GridSearchCV(estimator=model_log_reg,
                         param_grid=params_reg,
                         cv=10,
                         n_jobs=-1,
                         scoring='f1',
                         verbose=3
                         )

s = time.time()
model_l_r.fit(train_tf_idf, target_train)
e = time.time()
print(f'Логистическая регрессия')
print(f'Скорость обучения составила {round(e-s,2)} секунд')
print(f'Лучшие параметры: {model_l_r.best_params_}')
print(f'f1-мера по валидационной выборке: {round(model_l_r.best_score_,2)}')
#print(f'f1-мера по тестовой выборке: {round(f1(target_test, predictions_test),2)}')

Fitting 10 folds for each of 7 candidates, totalling 70 fits
Логистическая регрессия
Скорость обучения составила 106.93 секунд
Лучшие параметры: {'C': 4}
f1-мера по валидационной выборке: 0.72


### LightGBM classification

In [19]:
params_lgb = {
    'learning_rate': [.01,.03,.1,.15],
    'n_estimators': [100,500,1000],
}

model = lgb.LGBMClassifier(class_weight='balanced', random_state=12345,)
estimator=model,
model_lgb = GridSearchCV(estimator=model,
                         param_grid=params_lgb,
                         cv=5,
                         n_jobs=-1,
                         scoring='f1',
                         verbose=3
                         )

s = time.time()
model_lgb.fit(train_tf_idf, target_train)
e = time.time()
print('LGBM классификация')
print(f'Скорость обучения: {round(e-s,2)}')
print(f'Лучшие параметры: {model_lgb.best_params_}')
print(f'f1 для лучших параметров на валидационной выборке:', round(model_lgb.best_score_,2))
#print(f'f1 для лучших параметров на тестовой выборке:', round(f1(target_test, predictions_test),2))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
LGBM классификация
Скорость обучения: 278.24
Лучшие параметры: {'learning_rate': 0.1, 'n_estimators': 500}
f1 для лучших параметров на валидационной выборке: 0.7


Лучше всего показала себя модель логистической регрессии, обучим ее на тестовой выборке

In [34]:
model_log_reg = LogisticRegression(random_state=12345, max_iter=150, class_weight='balanced', C=4)

s = time.time()
model_log_reg.fit(train_tf_idf, target_train)
e = time.time()
print(f'Скорость обучения: {round(e-s,2)}')

s = time.time()
predictions_test = model_log_reg.predict(test_tf_idf)
e = time.time()
print(f'Скорость предсказания модели на тестовой выборке: {round(e-s,2)}')
print(f'f1 для лучших параметров на тестовой выборке:', round(f1(target_test, predictions_test),2))

Скорость обучения: 2.5
Скорость предсказания модели на тестовой выборке: 0.0
f1 для лучших параметров на тестовой выборке: 0.77


## Выводы

Были подготовлены данные по текстам комментариев для обучения, проанализированы два метода машинного обучения.

Установлено, что лучше себя показала модель логистической регрессии, f1 на тестовой выборке составила 0.77